<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Imports" data-toc-modified-id="Imports-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Imports</a></span></li><li><span><a href="#Constants" data-toc-modified-id="Constants-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Constants</a></span></li><li><span><a href="#Generate-stimulus" data-toc-modified-id="Generate-stimulus-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Generate stimulus</a></span></li><li><span><a href="#Plot" data-toc-modified-id="Plot-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Plot</a></span></li></ul></div>

The goal of this notebook, generate stimulus and posterior over the state for 50 trials for the discrimination and the prediction task, for 11 hazard rate values.

### Imports

In [1]:
import os, sys
sys.path.append(os.path.expanduser('~/Git/GitHub/work/Analysis_Audio2AFC_ChangePoint/Python_modules'))
import mmcomplexity as mmx
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import numpy as np

In [3]:
import pandas as pd

### Constants

In [4]:
num_trials = 50
h_vals = np.arange(0,11) / 10
print(h_vals)

[0.  0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1. ]


### Generate stimulus

In [5]:
list_of_df = []
for h in h_vals:
    # generate data
    sim_data = mmx.Audio2AFCSimulation(num_trials, [h], 0, [1], catch_rate=0)
    # sim_data.data.head()  # if you want to look at it

    # here we extract a StimulusBlock object to feed into the observer model
    sounds=list(sim_data.data['soundLoc'])
    sources=list(sim_data.data['sourceLoc'])
    stim_block = mmx.StimulusBlock(num_trials, h, sources=sources, sounds=sounds)
    # print(stim_block)

    # Now we create an observer model
    dm = mmx.KnownHazard(stim_block)
    dm.observe()  # we freeze the observations

    # we run the decision making algorithms for discrimination and prediction
    discrimination_decisions = list(dm.process())
    prediction_decisions = list(dm.process(filter_step=1))

    # turn log posterior odds into posterior over 'right' side
    try:
        discrimination_posterior = [mmx.log_odds_to_posterior(d[0])['right'] for d in discrimination_decisions]
        prediction_posterior = [mmx.log_odds_to_posterior(d[0])['right'] for d in prediction_decisions]
    except AssertionError:
        print('pb occurred at h-value ', h)
        raise

    # append posteriors to data frame
    sim_data.data['pred'] = prediction_posterior
    sim_data.data['discr'] = discrimination_posterior
    # sim_data.data.head()  # inspect if you wish
    list_of_df.append(sim_data.data)
final_df = pd.concat(list_of_df)

log odds inf too extreme, posterior set to delta
log odds inf too extreme, posterior set to delta
log odds inf too extreme, posterior set to delta
log odds inf too extreme, posterior set to delta
log odds inf too extreme, posterior set to delta
log odds inf too extreme, posterior set to delta
log odds inf too extreme, posterior set to delta
log odds inf too extreme, posterior set to delta
log odds inf too extreme, posterior set to delta


/home/adrian/Git/GitHub/work/Analysis_Audio2AFC_ChangePoint/Python_modules/mmcomplexity.py:229: RuntimeWarning: divide by zero encountered in double_scalars
  log_odds = np.log(posterior['right'] / posterior['left'])


In [6]:
print(final_df)

   sourceLoc  source_switch soundLoc  sound_switch  hazard  hazard_switch  \
0      right          False    right         False     0.0          False   
1      right          False    right         False     0.0          False   
2      right          False    right         False     0.0          False   
3      right          False    right         False     0.0          False   
4      right          False    right         False     0.0          False   
5      right          False     left          True     0.0          False   
6      right          False    right          True     0.0          False   
7      right          False    right         False     0.0          False   
8      right          False    right         False     0.0          False   
9      right          False    right         False     0.0          False   
10     right          False     left          True     0.0          False   
11     right          False    right          True     0.0          False   

### Plot

In [7]:
# plt.rcParams.update({'font.size': 22})
# fig = plt.figure(figsize=(18, 10))
# plt.plot(range(num_trials), sim_data.data['sourceLoc'])
# plt.plot(range(num_trials), sim_data.data['soundLoc'], 'o', markersize=8)
# plt.plot(range(num_trials), sim_data.data['pred'], linewidth=3, color='r')
# plt.plot(range(num_trials), sim_data.data['discr'], linewidth=3, color='b')
# plt.xlabel('trial number')
# plt.legend()
# plt.show

In [8]:
final_df.to_csv('data_to_test_models.csv')